In [1]:
stock_prices = []
with open("stock_prices.csv") as file:
    for line in file:
        tokens = line.split(",")
        day = tokens[0]
        price = float(tokens[1])
        stock_prices.append([day, price])

In [2]:
stock_prices

[['march 6', 310.0],
 ['march 7', 340.0],
 ['march 8', 380.0],
 ['march 9', 302.0],
 ['march 10', 297.0],
 ['march 11', 323.0]]

In [3]:
for entry in stock_prices:
    if entry[0] == "march 9":
        print(entry[1])

302.0


**Вышенаписанный способ - неэффективен. Сложность по времени - O(n)**

Словарь позводяет делать это со сложностью O(1)

In [4]:
stock_prices = {}
with open("stock_prices.csv") as file:
    for line in file:
        tokens = line.split(",")
        day = tokens[0]
        price = float(tokens[1])
        stock_prices[day] = price

In [5]:
stock_prices

{'march 6': 310.0,
 'march 7': 340.0,
 'march 8': 380.0,
 'march 9': 302.0,
 'march 10': 297.0,
 'march 11': 323.0}

In [6]:
stock_prices["march 9"]

302.0

Словарь - имлементация Hash Table/Map.


Хэш-таблица (или хэш-мап) — это структура данных, которая обеспечивает эффективный доступ к данным с помощью ключей. Она использует хэш-функции для преобразования ключей в индексы массива, что позволяет быстро вставлять, удалять и искать элементы.

Основные компоненты хэш-таблицы

**Массив (Bucket Array):**

Хэш-таблица содержит массив фиксированного размера, каждый элемент которого называется «корзиной» (bucket). В этой корзине будут храниться элементы, которые попадают на один и тот же индекс после хэширования.

**Хэш-функция:**

Функция, которая принимает ключ и возвращает индекс в массиве. Хорошая хэш-функция должна равномерно распределять ключи по массиву, чтобы минимизировать коллизии.

**Коллизии:**

Коллизия возникает, когда два разных ключа имеют одинаковый хэш-индекс. Хэш-таблицы должны эффективно решать коллизии.

**Методы обработки коллизий:**

1.*Открытая адресация (Open Addressing):*

Все элементы хранятся непосредственно в массиве, и при коллизии выполняется линейное или квадратичное пробирование для поиска следующей свободной ячейки.

2.*Цепочки (Chaining):*

Каждая корзина содержит указатель на связанный список или другую структуру данных, в которой хранятся все элементы, попавшие в одну корзину.

![](Screenshot_1.png)

![](Screenshot_2.png)

Таким же образом можно применить Hash Function ко всем остальным ключам.

**Пример Hash функции**

![](Screenshot_3.png)

Здесь используется ASCII код. Все ASCII идексы символов строки складываются. После берётся остаток от деления суммы на длину выделенного массива. В данном случае - это 609 % 10 = 9

**Complexities**

Lookup by key - O(1) on average

Insertion/Deletion - O(1) on average

In [7]:
def get_hash(key):
    h = 0
    for char in key:
        h += ord(char)
    return h % 100

In [8]:
get_hash("march 6")

9

In [9]:
class HashMapChMStr:
    """
    Chaining for collision handling; 
    Modulo from sum of a unicode codes of a key as a hash function;
    String only as a key.
    """
    # k - length of the key
    def __init__(self, size_max=16, load_factor_max=1.9):  # O(n)
        self.MAX = size_max
        self.LF_MAX = load_factor_max
        self.arr = [[] for bucket in range(self.MAX)]

    def get_hash(self, key):  # O(k)
        hash = 0
        for char in key:
            hash += ord(char)
        return hash % self.MAX

    def __setitem__(self, key, value):  # O(n) -> O(1) when key is short
        load_factor = self.count()/len(self.arr)
        if load_factor > self.LF_MAX:
            self.MAX = self.MAX * 2
            self._resize()
            
        bucket = self.arr[self.get_hash(key)]

        # Chaining
        found = False
        for idx, element in enumerate(bucket):
            if len(element) == 2 and element[0] == key:
                bucket[idx] = (key, value)
                found = True
                break
        if not found:
            bucket.append((key, value))

    def __delitem__(self, key):  # O(n) -> O(1) when key is short
        load_factor = self.count()/len(self.arr)
        if load_factor <= self.LF_MAX / 4 and self.count() > 10:
            self.MAX = self.MAX // 2
            
        bucket = self.arr[self.get_hash(key)]
        for idx, element in enumerate(bucket):
            if element[0] == key:
                del bucket[idx]
                return
        raise KeyError("Given key was not found")

    def __getitem__(self, key):  # O(k) -> O(1) when key is short
        for element in self.arr[self.get_hash(key)]:
            if element[0] == key:
                return element[1]
        raise KeyError("Given key was not found")

    def keys(self):  # O(mr); m - number of buckets, r - average length of a bucket
        keys = []
        for bucket in self.arr:
            for kv in bucket:
                keys.append(kv[0])
        return keys

    def values(self):  # O(mr); m - number of buckets, r - average length of a bucket
        values = []
        for bucket in self.arr:
            for kv in bucket:
                values.append(kv[1])
        return values

    def key_value_pairs(self):  # O(mr); m - number of buckets, r - average length of a bucket
        kv_pairs = []
        for bucket in self.arr:
            for kv in bucket:
                kv_pairs.append(kv)
        return kv_pairs

    def clear(self):  # O(n)
        self.arr = [[] for bucket in self.arr]

    def count(self):
        count = 0
        for bucket in self.arr:
            for kv in bucket:
                count += 1
        return count

    def _resize(self):
        kv_pairs = self.key_value_pairs()
        self.arr = [[] for bucket in range(self.MAX)]
        for pair in kv_pairs:
            hash = self.get_hash(pair[0])
            self.arr[hash].append(pair)
            

In [10]:
ht = HashMapChMStr(150)
# ht["1"] = 1
# ht["2"] = 2
# print(ht.key_value_pairs(), "|", ht.count())
# del ht["1"]
# print(ht.key_value_pairs(), "|", ht.count())
for num in range(500):
    ht[str(num)] = num
print(len(ht.arr))
ht.arr

300


[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('0', 0)],
 [('1', 1)],
 [('2', 2)],
 [('3', 3)],
 [('4', 4)],
 [('5', 5)],
 [('6', 6)],
 [('7', 7)],
 [('8', 8)],
 [('9', 9)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('10', 10)],
 [('11', 11), ('20', 20)],
 [('12', 12), ('21', 21), ('30', 30)],
 [('13', 13), ('22', 22), ('31', 31), ('40', 40)],
 [('14', 14), ('23', 23), ('32', 32), ('41', 41), ('50', 50)],
 [('15', 15), ('24', 24), ('33', 33), ('42', 42), ('51', 51), ('60', 60)],
 [('16', 16),
  ('25', 25),
  ('34', 34),
  ('43', 43),
  ('52', 52),
  ('61', 61),
  ('70', 70)],
 [('17', 17),
  ('26', 26),
  ('35', 35),
  ('44', 44),
  ('53', 5

In [11]:
# Creating and filling hash map
hash_table = HashMapChMStr()
hash_table['march 6'] = 310
hash_table['march 6'] = 120
hash_table['march 6'] = 154
hash_table['march 17'] = 120
hash_table['march 2'] = 120
hash_table['march 4'] = 193
hash_table.key_value_pairs()

[('march 6', 154), ('march 17', 120), ('march 2', 120), ('march 4', 193)]

In [12]:
# Accessing hash map items
print(hash_table['march 6'])
print(hash_table["march 17"])
print(hash_table['march 2'])
print(hash_table['march 4'])
# KeyError exception
# print(hash_table['march 4'])

154
120
120
193


In [13]:
# Deleting hash map items
del hash_table["march 6"]
# KeyError exception
# print(hash_table["march 6"])

In [14]:
hash_table.arr[:10]

[[], [], [], [('march 17', 120)], [], [], [], [], [], []]

**Обработка Коллизий**

## Chaining

Всякий элемент хэш-таблицы (её массива) есть ссылка на отдельный связный список, в который добавляются новые элементы при коллизии.

![](Screenshot_4.png)

В таком случае поиск элемента:

O(1) on avg, O(n) in worst case

## Linear Probing

Элементы всё также хранят значения, но в случае коллизии, будет пробироваться следующий индекс до тех пор, пока не найдётся пустой. Если достигнут конец массива, пробирование продолжается с его начального элемента.

![](Screenshot_5.png)


In [15]:
collisions = HashMapChMStr()
collisions["march 6"] = 120
collisions["march 17"] = 986

In [16]:
print(collisions.get_hash("march 6"))
print(collisions.get_hash("march 17"))

1
3


In [17]:
collisions["march 6"]

120

**Exercises**

In [18]:
# 1. nyc_weather.csv contains new york city weather for first few days in the month of January. Write a program that can answer following,
# What was the average temperature in first week of Jan
# What was the maximum temperature in first 10 days of Jan
# Figure out data structure that is best for this problem


def avg(values):
    """Time Complexity - O(n); Space Complexity - O(n)"""
    return sum(values)/len(values)


with open("nyc_weather.csv") as nyc_weather:
    lines = nyc_weather.readlines()
    temps = [float(val.split(',')[1].replace('\n', '')) for val in lines[1:]]
print(f"Length: {len(temps)}")

print(f"i. {round(avg(temps[:7]), 2)}")

print(f"ii. {max(temps)}")

Length: 10
i. 31.29
ii. 38.0


In [19]:
# 2. nyc_weather.csv contains new york city weather for first few days in the month of January. Write a program that can answer following,
# What was the temperature on Jan 9?
# What was the temperature on Jan 4?
# Figure out data structure that is best for this problem

# Here I used my hash map implementation

jan_weather_map = HashMapChMStr()
with open("nyc_weather.csv") as nyc_weather:
    lines = nyc_weather.readlines()[1:]
    for entry in lines:
        try:
            tokens = entry.split(',')
            jan_weather_map[tokens[0]] = int(tokens[1])
        except ValueError:
            print("Invalid value for integer conversion. This value will be skipped.")
            
print(f"i. {jan_weather_map['9-Jan']}")

print(f"ii. {jan_weather_map['4-Jan']}")

i. 35
ii. 34


In [20]:
# 3. poem.txt Contains famous poem "Road not taken" by poet Robert Frost. You have to read this file in python and print every word and its count 
# as show below. Think about the best data structure that you can use to solve this problem and figure out why you selected that specific data structure.

# Here i used dictionary
import re

word_count = {}
with open('poem.txt') as poem:
    lines = [re.sub("[!;:,.-]", "", line).replace("\n", "").split(" ") for line in poem.readlines()]

words = []
for line in lines:
    words += line

for word in words:
    if word not in word_count:
        word_count[word.lower()] = 1
    else:
        word_count[word.lower()] += 1

word_count

{'two': 1,
 'roads': 2,
 'diverged': 2,
 'in': 2,
 'a': 3,
 'yellow': 1,
 'wood': 2,
 'and': 1,
 'sorry': 1,
 'i': 1,
 'could': 2,
 'not': 1,
 'travel': 1,
 'both': 2,
 'be': 2,
 'one': 3,
 'traveler': 1,
 'long': 1,
 'stood': 1,
 'looked': 1,
 'down': 1,
 'as': 5,
 'far': 1,
 'to': 2,
 'where': 1,
 'it': 2,
 'bent': 1,
 'the': 8,
 'undergrowth': 1,
 '': 3,
 'then': 1,
 'took': 2,
 'other': 1,
 'just': 1,
 'fair': 1,
 'having': 1,
 'perhaps': 1,
 'better': 1,
 'claim': 1,
 'because': 1,
 'was': 1,
 'grassy': 1,
 'wanted': 1,
 'wear': 1,
 'though': 1,
 'for': 2,
 'that': 3,
 'passing': 1,
 'there': 1,
 'had': 2,
 'worn': 1,
 'them': 1,
 'really': 1,
 'about': 1,
 'same': 1,
 'morning': 1,
 'equally': 1,
 'lay': 1,
 'leaves': 1,
 'no': 1,
 'step': 1,
 'trodden': 1,
 'black': 1,
 'oh': 1,
 'kept': 1,
 'first': 1,
 'another': 1,
 'day': 1,
 'yet': 1,
 'knowing': 1,
 'how': 1,
 'way': 2,
 'leads': 1,
 'on': 1,
 'doubted': 1,
 'if': 1,
 'should': 1,
 'ever': 1,
 'come': 1,
 'back': 1,
 'shal

In [42]:
# 4. Implement hash table where collisions are handled using linear probing. We learnt about linear probing in the video tutorial. 
# Take the hash table implementation that uses chaining and modify methods to use linear probing. Keep MAX size of arr in hashtable as 10.

class HashMapLpMStr:
    """
    Linear Probing for collision handling; 
    Modulo from sum of a unicode codes of a key as a hash function;
    String only as a key.
    """
    # k - length of the key
    def __init__(self, size_max=10, load_factor_max=1.9):  # O(n)
        self.MAX = size_max
        self.LF_MAX = load_factor_max
        self.arr = [None for bucket in range(self.MAX)]

    def get_index(self, key):
        circle = self._get_hash(key)
        probe = self._get_hash(key)
        while True:
            if not self.arr[probe]:
                return probe
            else:
                if probe == len(self.arr):
                    probe = 0
                    continue
                if probe == circle:
                    raise Exception("Hash map is full")
                probe += 1

    def __setitem__(self, key, value):  # O(n) -> O(1) when key is short
        load_factor = self.count()/len(self.arr)
        if load_factor >= self.LF_MAX:
            self.MAX = self.MAX * 2
            self._resize()

        idx = self.get_index(key)
        self.arr[idx] = (key, value)
        

    def __delitem__(self, key):  # O(n) -> O(1) when key is short
        load_factor = self.count()/len(self.arr)
        if load_factor <= self.LF_MAX / 4 and self.count() > 10:
            self.MAX = self.MAX // 2
            
        del self.arr[self.get_index(key)]

    def __getitem__(self, key):  # O(k) -> O(1) when key is short
        return self.arr[self.get_index(key)]

    def keys(self):  # O(mr); m - number of buckets, r - average length of a bucket
        keys = []
        for bucket in self.arr:
            append(kv[0])
        return keys

    def values(self):  # O(mr); m - number of buckets, r - average length of a bucket
        values = []
        for bucket in self.arr:
            values.append(kv[1])
        return values

    def key_value_pairs(self):  # O(mr); m - number of buckets, r - average length of a bucket
        kv_pairs = []
        for bucket in self.arr:
            if bucket:
                kv_pairs.append(bucket)
        return kv_pairs

    def clear(self):  # O(n)
        self.arr = [None for bucket in self.arr]

    def count(self):
        count = 0
        for _ in self.arr:
            if _:
                count += 1
        return count

    def _resize(self):
        kv_pairs = self.key_value_pairs()
        self.arr = [None for bucket in range(self.MAX)]
        for pair in kv_pairs:
            idx = self.get_index(pair[0])
            arr[idx] = pair

    def _get_hash(self, key):  # O(k)
        hash = 0
        for char in key:
            hash += ord(char)
        return hash % self.MAX
        

In [45]:
test = HashMapLpMStr()
for _ in range(10):
    test[str(_)] = _
test.arr

[('2', 2),
 ('3', 3),
 ('4', 4),
 ('5', 5),
 ('6', 6),
 ('7', 7),
 ('8', 8),
 ('9', 9),
 ('0', 0),
 ('1', 1)]